# Data and Libraries

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
import pmdarima as pm
from pmdarima import model_selection

In [2]:
# Load the Data
# YYYY-MM-DD
df = pd.read_csv('../nyc_data.csv', index_col=0, parse_dates=True)
df.head()

,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [3]:
# Rename Variables
df = df.rename(columns={'Demand': 'y'})
df.head()

,y,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,,
2015-01-01,720.000885,0,0,0,3.68,41.305
2015-01-02,581.276773,0,0,0,4.73,131.574
2015-01-03,754.117039,0,0,0,7.23,162.700
2015-01-04,622.252774,0,0,0,10.96,160.281
2015-01-05,785.373319,0,0,0,6.92,51.077


In [5]:
# Extract the regressors
X = df.iloc[:, 1:]
X.head(0)

,Easter,Thanksgiving,Christmas,Temperature,Marketing
Date,,,,,


# Stationarity

In [8]:
# Test
from statsmodels.tsa.stattools import adfuller
p = adfuller(x = df.y)[1]
# Condition to read test
if p < 0.05:
    print(f"The series is Stationary with a value of {p}")
else:
    print(f"Not Stationary with a value of {p}")

Not Stationary with a value of 0.37677707077290995


In [10]:
# Differencing
df.y.diff().dropna()

# Test
p = adfuller(x = df.y.diff().dropna())[1]
# Condition to read test
if p < 0.05:
    print(f"The series is Stationary with a value of {p}")
else:
    print(f"Not Stationary with a value of {p}")

The series is Stationary with a value of 3.3557739456282398e-22


# SARIMAX Model

In [13]:
# Model
# For the last value of the seasonal_order parameter
# Hourly : 24, Daily: 7, Weekly: 52, monthly: 12, Quarterly: 4
model = pm.ARIMA(order=(1,1,1), 
                 seasonal_order=(1,1,1, 7),
                 x = X,
                 suppress_warnings=True,
                 force_stationarity=False)

# Cross Validation

In [15]:
# Cross-Validation
cv = model_selection.RollingForecastCV(h = 31,
                                       step = 16,
                                       initial=df.shape[0] - 180)

cv_score = model_selection.cross_val_score(model, 
                                           y = df.y,
                                           scoring='mean_squared_error',
                                           cv = cv,
                                           verbose=2,
                                           error_score=1000000000)

[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ..........................................................
[CV] fold=4 ..........................................................
[CV] fold=5 ..........................................................
[CV] fold=6 ..........................................................
[CV] fold=7 ..........................................................
[CV] fold=8 ..........................................................
[CV] fold=9 ..........................................................


In [18]:
# CV Performance
error = np.sqrt(np.average(cv_score))
error

59.95525756014244

# Parameter Tunining

In [20]:
# Grid
param_grid = {'p': [0,1],
              'd': [1],
              'q': [0,1],
              'P': [0,1],
              'D': [0,1],
              'Q': [0,1]}

grid = ParameterGrid(param_grid=param_grid)
len(grid)

32

In [27]:
# Parameter loop
rmse = []

# Loop
for index, param in enumerate(grid):

    print(f"Loop No. {index}", end= '\n**************\n')

    # Build model
    model = pm.ARIMA(order=(param['p'], param['d'], param['q']),
                     seasonal_order= (param['P'], param['D'], param['Q'], 7),
                     x = X,
                     suppress_warnings=True,
                     force_stationarity=True)
    
    # CV
    cv = model_selection.RollingForecastCV(h = 31,
                                           step=16,
                                           initial=df.shape[0] - 180)
    
    cv_score = model_selection.cross_val_score(model,
                                               y = df.y,
                                               scoring='mean_squared_error',
                                               cv=cv,
                                               verbose=0,
                                               error_score=100000000)
    
    error = np.sqrt(np.average(cv_score))
    rmse.append(error)

Loop No. 0
**************
Loop No. 1
**************
Loop No. 2
**************
Loop No. 3
**************
Loop No. 4
**************
Loop No. 5
**************
Loop No. 6
**************
Loop No. 7
**************
Loop No. 8
**************
Loop No. 9
**************
Loop No. 10
**************
Loop No. 11
**************
Loop No. 12
**************
Loop No. 13
**************
Loop No. 14
**************
Loop No. 15
**************
Loop No. 16
**************
Loop No. 17
**************
Loop No. 18
**************
Loop No. 19
**************
Loop No. 20
**************
Loop No. 21
**************
Loop No. 22
**************
Loop No. 23
**************
Loop No. 24
**************
Loop No. 25
**************
Loop No. 26
**************
Loop No. 27
**************
Loop No. 28
**************
Loop No. 29
**************
Loop No. 30
**************
Loop No. 31
**************


# Find best Parameter

In [30]:
tuning_results = pd.DataFrame(grid)
tuning_results['rmse'] = rmse
tuning_results

,D,P,Q,d,p,q,rmse
0,0,0,0,1,0,0,109.196397
1,0,0,0,1,0,1,86.990889
2,0,0,0,1,1,0,88.724335
3,0,0,0,1,1,1,85.350878
4,0,0,1,1,0,0,111.422775
5,0,0,1,1,0,1,84.774377
6,0,0,1,1,1,0,91.853986
7,0,0,1,1,1,1,83.556192
8,0,1,0,1,0,0,134.711130
9,0,1,0,1,0,1,93.974142


In [31]:
# Find best params
tuning_results.iloc[np.argmin(rmse)]


D        1.000000
P        1.000000
Q        1.000000
d        1.000000
p        1.000000
q        1.000000
rmse    59.955258
Name: 31, dtype: float64

In [32]:
# Export
best_param = tuning_results[tuning_results.rmse == tuning_results.rmse.min()].transpose()
best_param.to_csv('best_params_ARIMA.csv')